In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 138.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 104.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [2]:
import pandas as pd
import gc
import numpy as np
import torch
import soundfile as sf
import librosa
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoProcessor, 
    AutoModelForAudioClassification,
    TrainingArguments, 
    Trainer
)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import os

# Загрузка данных
train = pd.read_csv('/kaggle/input/vseros-audio/train_dataset.csv')
train['path'] = '/kaggle/input/vseros-audio/train_data/train_opus/audio/' + train['id'] + '.opus'


# Выбираем подмножество для быстрого тестирования
# train, _ = train_test_split(train, train_size=45_000, random_state=42, stratify=train['label'])

print(f"Размер тренировочных данных: {len(train)}")
print(f"Распределение классов: {train['label'].value_counts()}")

2025-10-19 05:32:53.470879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760851973.619500      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760851973.664993      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Размер тренировочных данных: 90000
Распределение классов: label
0    45000
1    45000
Name: count, dtype: int64


In [3]:
# Создаем Dataset
ds = Dataset.from_dict({
    'path': train['path'].tolist(),
    'label': train['label'].tolist()
})

# Загружаем процессор
model_id = "openai/whisper-small"  # Используем base версию для экономии памяти
processor = AutoProcessor.from_pretrained(model_id)

def preprocess_function(examples):
    """Функция предобработки аудио"""
    audio_arrays = []
    
    for file_path in examples["path"]:
        try:
            # Загружаем аудио и преобразуем к нужной частоте дискретизации
            audio_array, sampling_rate = sf.read(file_path)
            
            # Преобразуем к моно если нужно
            if len(audio_array.shape) > 1:
                audio_array = np.mean(audio_array, axis=1)
            
            # Ресемплинг до 16kHz если нужно
            if sampling_rate != 16000:
                audio_array = librosa.resample(audio_array, orig_sr=sampling_rate, target_sr=16000)
            
            audio_arrays.append(audio_array)
            
        except Exception as e:
            print(f"Ошибка при обработке файла {file_path}: {e}")
            # Добавляем нулевой массив в случае ошибки
            audio_arrays.append(np.zeros(16000))
    
    # Обрабатываем аудио через процессор
    inputs = processor(
        audio_arrays, 
        sampling_rate=16000,
        # padding=True,
        max_length=16000 * 30,  # Максимум 10 секунд
        # truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    return inputs

# Предобрабатываем данные
print("Начинаем предобработку аудио...")
dataset_processed = ds.map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=['path'],
    # num_proc=
)

# Переименовываем колонки для Trainer
dataset_processed = dataset_processed.rename_column("label", "labels")
dataset_processed.set_format("torch")

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Начинаем предобработку аудио...


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

In [4]:
# Разделяем на train/validation
datasets = dataset_processed.train_test_split(seed=56, test_size=0.05)
print(f"Train size: {len(datasets['train'])}, Test size: {len(datasets['test'])}")

Train size: 85500, Test size: 4500


In [5]:
# Функция для вычисления метрик
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    
    # Вычисляем FAR и FRR
    tp = np.sum((predictions == 1) & (labels == 1))
    fp = np.sum((predictions == 1) & (labels == 0))
    fn = np.sum((predictions == 0) & (labels == 1))
    tn = np.sum((predictions == 0) & (labels == 0))
    
    num_pos = np.sum(labels == 1)
    num_neg = np.sum(labels == 0)
    
    frr = fn / num_pos if num_pos > 0 else 0
    far = fp / num_neg if num_neg > 0 else 0
    
    tpr = 1 - frr  # True Positive Rate
    tnr = 1 - far  # True Negative Rate
    
    # Harmonic mean of TPR and TNR
    if tpr > 0 and tnr > 0:
        score = 2 * (tpr * tnr) / (tpr + tnr)
    else:
        score = 0
    
    return {
        'accuracy': acc,
        'f1': f1,
        'score': score,
    }

# Аргументы для тренировки
training_args = TrainingArguments(
    output_dir='./hubert_audio_classification',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=6,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="score",
    greater_is_better=True,
    # fp16=True,
    fp16=True,
    # bf16=True,
    dataloader_pin_memory=False,
    save_total_limit=1,
    report_to='none',
    seed=0
)

# Загружаем модель
print("Загружаем модель...")
model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=2,
    ignore_mismatched_sizes=True,
    # torch_dtype=torch.bfloat16,
)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['test'],
    tokenizer=processor,
    compute_metrics=compute_metrics
)

# Запускаем тренировку
print("Начинаем тренировку...")
train_results = trainer.train()

Загружаем модель...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at openai/whisper-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_19/4197453059.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Начинаем тренировку...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:461: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50

In [6]:
print("Инференс...")

Инференс...


In [7]:
test_files = pd.Series([x for x in os.listdir('/kaggle/input/vseros-audio/test_data/test_opus/audio') if not x.startswith('._')])

In [8]:
test_paths = '/kaggle/input/vseros-audio/test_data/test_opus/audio/' + test_files

In [9]:
test_ds = Dataset.from_dict({
    'path': test_paths.tolist()
})

In [10]:
del datasets
gc.collect()

99

In [11]:
test_ds = test_ds.map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=['path'],
    # num_proc=
)
test_ds.set_format("torch")

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

In [12]:
preds = trainer.predict(test_ds)

In [13]:
idxs = test_files.str.replace('.opus', '')

In [14]:
sub = pd.DataFrame({'id': idxs, 'label': preds.predictions.argmax(1)})
sub.head()

id  label
0  3292593169695156298812886838190965856838      0
1  6556265671716347144565922071836364658607      0
2  9262812783002930407390476396593829992874      0
3  3943694282842649998713806087991437332498      0
4  3045360991217095260474965081416433033575      0

In [15]:
sub.to_csv('sub_audio_whisper-small-fd.csv', index=False)

In [16]:
try:
    np.save('whisper-small-fd_logits.npy', preds.predictions)
except:
    pass

In [17]:
sub['label'].value_counts()

label
1    13588
0    13412
Name: count, dtype: int64